<img align=right src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=150>
<img src=images/sglogo.png width=50 align="left">

# Vault Authentication Methods
<img src="images/vault-auth-1.png" width=600>  


Vault supports many different authentication methods, suitable for human and applications (e.g. AppRole, AWS/Azure/Google, K8s).
Most Vault auth methods need to be explicitly enabled.  Each method has a default path.


**Prerequisites:** 
You need to have Vault up and running

`VAULT_UI=true VAULT_REDIRECT_ADDR=http://127.0.0.1:8200 vault server -log-level=trace -dev -dev-root-token-id=root -dev-listen-address=127.0.0.1:8200 -dev-ha -dev-transactional`

In [ ]:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_SKIP_VERIFY=true

In [ ]:
vault login root

In [ ]:
vault status; vault version

---
# Userpass Authentication method
authenticates users with usernames and passwords managed by Vault.



### Setup

In [ ]:
vault auth enable userpass

**Configure username/password**

In [ ]:
vault auth enable userpass
vault write auth/userpass/users/arthur password="passw0rd" policies="static-policy"
vault write auth/userpass/users/samantha password="passw0rd" policies="static-policy"
vault write auth/userpass/users/tio password="passw0rd" policies="static-policy, dynamic-policy"
vault write auth/userpass/users/moayad password="passw0rd" policies="static-policy, dynamic-policy"
vault write auth/userpass/users/raymond password="passw0rd" policies="static-policy, dynamic-policy"

### Login

In [ ]:
unset VAULT_TOKEN
vault login -method=userpass username=tio password=passw0rd

**Via cURL**
The response will contain the token at `auth.client_token`

In [ ]:
curl -s --request POST --data '{"password": "passw0rd"}' \
    http://127.0.0.1:8200/v1/auth/userpass/login/tio | jq

In [ ]:
vault list auth/userpass/users

---
# Github Authentication method

### Setup Vault-Github integration

In [ ]:
export VAULT_ADDR=http://127.0.0.1:8200
vault login root

In [ ]:
vault auth enable github

### Configure Vault to talk to Github

In [ ]:
vault write auth/github/config organization=hashicorp

### Map users/teams of that Github organization to policies in Vault.  
Team names must be "slugified"

In [ ]:
vault write auth/github/map/teams/team-se value=static-policy

In [ ]:
vault write auth/github/map/users/tiobagio value=dynamic-policy

### Login using Github token
The `github` auth method can be used to authenticate with Vault using a Github personal access token.  This method of authentication is most useful for operators or developers usign Vault directly.

In [ ]:
MY_GITHUB_TOKEN=$(cat /tmp/githubtoken)
vault login -method=github token=$MY_GITHUB_TOKEN

**Using cURL**

In [ ]:
MY_GITHUB_TOKEN=$(cat /tmp/githubtoken)
curl -s --request POST --data " {\"token\": \"$MY_GITHUB_TOKEN\"} " \
    http://127.0.0.1:8200/v1/auth/github/login  | jq

---
# AWS Auth method
There are two authentication types in the AWS auth method:  `iam` and `ec2`.  

The aws auth method provides an automated mechanism to retrieve Vault token for IAM principals and AWS EC2 instances.   
This method does not require manual first-deploying or profisioning security-sensitive credentials.

### Setup AWS auth config

In [ ]:
export VAULT_ADDR=http://127.0.0.1:8200
vault login root

In [ ]:
vault auth enable aws

In [ ]:
source ~/.zshrc; echo $AWS_ACCESS_KEY_ID; aws sts get-caller-identity

In [ ]:
# tbagio profile in ~/.aws/credentials
#
vault write auth/aws/config/client \
  secret_key=$AWS_SECRET_ACCESS_KEY \
  access_key=$AWS_ACCESS_KEY_ID 

In [ ]:
vault read auth/aws/config/client

**EC2 authentication type**

In [ ]:
VAULT_TOKEN=root vault write auth/aws/role/dev-role-ec2 \
   auth_type=ec2 \
   bound_ami_id=ami-fce3c696 \
   policies=prod,dev max_ttl=5m

&nbsp;   
**IAM authentication type**

In [ ]:
export VAULT_ADDR=http://127.0.0.1:8200
vault login root

In [ ]:
#bound_iam_principal_arn=arn:aws:iam::451256746182:user/tbagio \
#bound_iam_role_arn=arn:aws:iam::451256726182:role/example-role \
VAULT_TOKEN=root vault write auth/aws/role/dev-role-iam \
   auth_type=iam \
   bound_iam_principal_arn="arn:aws:iam::451256726182:*"\
   policies=dynamic-policy,prod,dev ttl=5m

In [ ]:
vault write auth/aws/config/client iam_server_id_header_value=vault.example.com

In [ ]:
vault read auth/aws/role/dev-role-iam

### Login using AWS IAM information

In [ ]:
# chip
vault login -method=aws header_value=vault.example.com \
  role=dev-role-iam \
  aws_access_key_id=$(cat /tmp/aws_access_key) \
  aws_secret_access_key=$(cat /tmp/aws_secret_key)

In [ ]:
AWS_SECRET_ACCESS_KEY=$(cat /tmp/aws_secret_key); AWS_ACCESS_KEY_ID=$(cat /tmp/aws_access_key); aws sts get-caller-identity

### and get a secret

In [ ]:
VAULT_TOKEN=s.7N3PlDOxaVd7llNInH7vBPFW vault  kv get secret1/training

---
# Azure Auth 

In [ ]:
vault login root
vault auth enable azure

In [ ]:
vault write auth/azure/config \
    resource=https://vault.hashicorp.com \
    tenant_id=7cd1f227-ca67-4fc6-a1a4-9888ea7f388c \
    client_id=dd794de4-4c6c-40b3-a930-d84cd32e9699 \
    client_secret=IT3B2XfZvWnfB98s1cie8EMe7zWg483Xy8zY004=

In [ ]:
vault write auth/azure/role/dev-role \
    policies="prod,dev" \
    bound_subscription_ids=6a1d5988-5917-4221-b224-904cd7e24a25 \
    bound_resource_groups=vault

### Login

In [ ]:
vault write auth/azure/login \
   role="dev-role" \
   jwt="eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9..." \
   subscription_id="12345-..." \
   resource_group_name="test-group" \
   vm_name="test-vm"

In [ ]:
vault write auth/azure/login \
   role="dev-role" \
   jwt="$(curl -s 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=2018-02-01&resource=https%3A%2F%2Fvault.hashicorp.com%2F' -H Metadata:true | jq -r '.access_token')" \
   subscription_id=$(curl -s -H Metadata:true "http://169.254.169.254/metadata/instance?api-version=2017-08-01" | jq -r '.compute | .subscriptionId')  \
   resource_group_name=$(curl -s -H Metadata:true "http://169.254.169.254/metadata/instance?api-version=2017-08-01" | jq -r '.compute | .resourceGroupName') \
   vm_name=$(curl -s -H Metadata:true "http://169.254.169.254/metadata/instance?api-version=2017-08-01" | jq -r '.compute | .name')

---
# Google Cloud Auth Method
The `gcp` auth method allows Google Cloud Platform entities to authenticate to Vault.  
Vault treats Google Cloud as a trusted third party and verifies authenticating entities against the Google Cloud APIs.
  
This backend allow for authentication of:
* IAM service accounts
* GCE (compute engine) instances

It does not support authenticating arbitrary Google/G Suite users or generic OAuth against Google.

### Setup

In [ ]:
vault login root
vault auth enable gcp

In [ ]:
vault write auth/gcp/config credentials=@/path/to/credentials.json

**Create a role**  
For `iam` type role.  Note that `bound_service_account` is only required for `iam`-type roles

In [ ]:
vault write auth/gcp/role/my-iam-role \
   type="iam" \
   policies="dev, prod" \
   bound_service_accounts="my-service@my-project.iam.gserviceaccount.com"

For `gce` type role

In [ ]:
vault write auth/gcp/role/my-gce-role \
   type="gce" \
   policies="dev, prod" \
   bound_projects="my-prject1,my-project2" \
   bound_zones="us-east1-b" \
   bound_labels="fooLbar, zip:zap" \
   bound_service_accounts="my-service@my-project.iam.gserviceaccount.com"

### Login/Authenticate

In [ ]:
vault write -field=token auth/gcp/login \
  role="my-role" \
  jwt="..." 

Vault includes a CLI helper that obtains a signed JWT locally and sends the request to Vault.  This helper is only for IAM-type roles.

In [ ]:
vault login -method=gcp \
   role="my-role" \
   service_account="authenticating-account@my-project.iam.gserviceaccount.com" \
   project="my-project" \
   jwt_exp="15m" \
   credentials=@path/to/signer/credentials.json

---
# Okta Auth method

### Configure Vault to talk to Okta

In [ ]:
export VAULT_ADDR=http://127.0.0.1:8200
vault login root
vault auth enable okta

In [ ]:
vault write auth/okta/config \
  base_url="okta.com" \
  org_name="hashicorp" \
  token_policies="static-policy"

**Anyone who successfully authenticates via Okta who's a member of scientists" group will receive a Vault token with "nuclear-reactor" policy attached**

In [ ]:
vault write auth/okta/users/tio@hashicorp.com policies="dynamic-policy,other-policy"

### Login using Okta method

In [ ]:
vault login -method=okta username=tio@hashicorp.com

**Using cURL**

In [ ]:
MY_PASSWORD=$(cat /tmp/mypassword)
curl -s --request POST --data " {\"password\": \"$MY_PASSWORD\"} " \
    http://127.0.0.1:8200/v1/auth/okta/login/tio@hashicorp.com

# AppRole

Vault provides internal and external authentication methods.  External methods are called *tusted third-party authenticators* such as AWS, LDAP, Github, etc.

In some situations trusted third-party authenticator is not available, so Vault has an alternate approach - **AppRole** .   
AppRole allows machines or apps to authenticate.  This auth method is oriented to automated workflows (machines and services) and is less useful for human operators.

* AppRole generates **RoleID** and **SecretID**
    - RoleID is an identifier similar to username
    - SecretID is a credential to login (similart to password)   
    

### Setup
<img src="images/vault-approle-workflow.png" width=800>


In [ ]:
vault login root
vault auth enable approle

In [ ]:
vault write auth/approle/role/tio-crm \
  token_num_users=3 \
  token_ttl=10m \
  token_max_ttl=30m \
  secret_id_ttl=5m \
  secret_id_num_uses=40 

### RoleID
RoleID is a unique identifier for the app.  They can be set to particular values to match introspected information by the client (ex: client's domain name).

In [ ]:
vault read auth/approle/role/tio-crm/role-id

### SecretID
SecretID is like a password, it is intended to always be secret.  SecretIDs can be created agains an AppRole either via generation of a 128-bit purely random UUID by the role itself or via specific custom values.  SecretIDs have properties like usage-limit, TTLs, and expirations.

**Pull and Push SecretID Modes**  
If SecretID is fetched from an AppRole, this is Pull Mode.  Most cases Pull mode is the better approach.

<img src="images/vault-approle-pull.png" width=800>  

If a "custom" secretID is set by the client, it is Push Mode.  
<img src="images/vault-approle-push.png" width=800>

Push Mode requires some other system to have knowledge of the full set of credentials (RoleID and SecretID) in order to create the entry.   
However in Pull Mode, the SecretID can be kept confidential from all parties except for the final authenticating client by using Response w Wrapping.
  


In [ ]:
vault write -f auth/approle/role/tio-crm/secret-id

### Login

In [ ]:
vault write auth/approle/login \
  role_id=d56c9c46-7198-4238-fb0c-3a2b26e97e18   \
  secret_id=e151cb41-0152-8974-e259-75d9fd7c2972 

**Using cURL**  
The default endpoint is `auth/approle/login`.  The response will contain the token at `auth.client_token`

In [ ]:
curl -s --request POST \
    --data "{\"role_id\":\"d56c9c46-7198-4238-fb0c-3a2b26e97e18\", \"secret_id\":\"e151cb41-0152-8974-e259-75d9fd7c2972\"}" \
    http://127.0.0.1:8200/v1/auth/approle/login |jq

---
### View Vault via a browser
Open a web browser and visit http://127.0.0.1:8200/ui/vault
Use **TOKEN** and **Sign in**

### TODO:
- how to set AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS for a certain profile

&nbsp;

---
#### Thank you.
<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=100 align="left">